In [1]:
import pandas as pd

from HestonModel import Heston, HestonPriceFunction

heston = Heston()
heston.load_data('Data/20230912_PLIQ_IP.csv')
heston.preprocess_data()
heston.get_from_market()
heston.optimize_params()


TypeError: Heston.optimize_params.<locals>.objective_function() missing 5 required positional arguments: 'market_price', 'strike', 'maturity', 'risk_free_rate', and 'option_type'

In [4]:
heston.clean.Pliq.std()

5390.266738228924

In [2]:
strike_price = 100
spot_price = 105
yearly_historical_volatility = 0.1
riskfree_rate = 0.0
kappa = 2
epsilon = 0.1   
rho = 0.0   

HestonPriceFunction(strike_price=strike_price, spot_price=spot_price, yearly_historical_volatility = yearly_historical_volatility,
                risk_free_rate=riskfree_rate, kappa = kappa, theta=yearly_historical_volatility**2, epsilon = epsilon, rho = rho, current = "2020/8/4", maturity = "2021/8/3",
                call_option = True)

7.027599518261383

In [6]:
import pandas as pd
import QuantLib as ql

def HestonPriceFunction(strike_price: float, spot_price: float, yearly_historical_volatility: float, 
                        risk_free_rate: float, kappa: float, epsilon: float, rho: float,
                        theta: float, current: str, time_to_maturity: float, call_option: bool, 
                        dividend_rate: float = 0.0, step: float = 0.001, runs: int = 1000):

    # Parameters
    variance = yearly_historical_volatility ** 2  # Initial variance is square of volatility
    option_type = ql.Option.Call if call_option else ql.Option.Put
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)

    # Current Date
    current_date = pd.to_datetime(current)
    
    # Calculate Maturity Date based on time to maturity
    maturity_date = current_date + pd.Timedelta(days=365 * time_to_maturity)
    
    # QuantLib uses Date objects
    valuation_date = ql.Date(current_date.day, current_date.month, current_date.year)
    maturity_ql_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year)
    ql.Settings.instance().evaluationDate = valuation_date

    # Exercise function takes maturity date of the option as input
    exercise = ql.EuropeanExercise(maturity_ql_date)
    option = ql.VanillaOption(payoff, exercise)

    # Spot price as a Quote object
    initial_value = ql.QuoteHandle(ql.SimpleQuote(spot_price))

    # Setting up flat risk-free and dividend yield curves
    day_count = ql.Actual365Fixed()
    risk_free_curve = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, risk_free_rate, day_count))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, dividend_rate, day_count))

    # Setting up the Heston process and model
    heston_process = ql.HestonProcess(risk_free_curve, dividend_yield, initial_value, variance, kappa, theta, epsilon, rho)
    heston_model = ql.HestonModel(heston_process)
    
    # Engine for pricing
    engine = ql.AnalyticHestonEngine(heston_model, step, runs)
    option.setPricingEngine(engine)

    # Calculating the option price
    price = option.NPV()
    return price


In [ ]:
import numpy as np
from scipy.optimize import basinhopping
import pandas as pd
import QuantLib as ql



def objective_function(params):
    v0, kappa, theta, epsilon, rho = params
    errors = []

    for market_price, strike, maturity in zip(market_prices, strikes, maturities):
        model_price = HestonPriceFunction(strike, spot_price, np.sqrt(v0), risk_free_rate, kappa,
                                          epsilon, rho, theta, current_date, maturity, True)
        error = (model_price - market_price) ** 2
        errors.append(error)

    return np.sum(errors)

# Initial guess for the parameters based on historical volatility
yearly_historical_volatility = 0.2  # Placeholder for historical volatility
initial_params = [yearly_historical_volatility ** 2, 2.0, 0.04, 0.1, -0.7]  # Initial parameter guess

# Define bounds for the parameters
bounds = [(0.001, 1), (0.1, 10), (0.001, 1), (0.1, 1), (-1, 1)]

# The basinhopping algorithm
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
result = basinhopping(objective_function, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)

# Results
optimized_params = result.x
minimum_error = result.fun

print("Optimized parameters:", optimized_params)
print("Minimum error:", minimum_error)
